# 0. 들어가기 앞서

* 'AS':0, '주문':1, '배송':2, '업무처리':3, '교환':4, '반품':5, '결제':6
* 참고: https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab

* 한국어언어모델 다양하게 사용해보기 : https://littlefoxdiary.tistory.com/81

# 1. 라이브러리 로드

In [1]:
# !pip install gluonnlp
# !pip install mxnet
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [3]:
# !pip install numpy --upgrade

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
##GPU 사용 시
device = torch.device("cuda:0")

# 2. 모델, 사전, 데이터셋 불러오기

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/adminuser/notebooks/modeling/question/[0차] KoBERT_cat_baseline/.cache/kobert_v1.zip
using cached model. /home/adminuser/notebooks/modeling/question/[0차] KoBERT_cat_baseline/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
import os
os.listdir("../[0차] 원본_preprocessing")

['.ipynb_checkpoints',
 '질의응답_K쇼핑_질문유형분류_원본.csv',
 '[0차] 원본데이터_preprocessing.ipynb',
 '질의응답_K쇼핑_질문분류_원본.csv']

In [10]:
df= pd.read_csv("../[0차] 원본_preprocessing/질의응답_K쇼핑_질문유형분류_원본.csv", index_col=0)

In [12]:
df.sample(10)

,msg,cat
934121,그때 방송을 늦게 봐가지고 좀 설명 듣고 주문하고 싶거든요.,1
501430,경기도 양평군 ㅇㅇㅇ ㅇㅇ ㅇㅇ 아파트 ㅇㅇㅇ동 ㅇㅇㅇ호 예요 이건 아까 자동 주...,2
341599,아 죄송한데 맞교환 좀 해주시면 안 될까요? 여름이 다 지나갈 것 같은데 못 입으면...,4
492669,그거를 자동으로 주문하는데 계속 안 되네요. 우리 아들한테 좀 부칠려고 하는데 어떻...,2
938096,물기는 안 생겨요?,1
725641,아 그럼 다시 휴대폰으로 주문할까요?,3
705619,아 무통장 입금 할게요.,3
28784,아 그러면 무통장 신한 은행 이걸로 되어 있는데 맞나요?,6
402363,그러면 맞교환이 되는 겁니까?,5
697065,네. 그런데 이 종은 반품을 하신다고요?,3


# 3. 질문유형 분류 시작

* 'AS':0, '주문':1, '배송':2, '업무처리':3, '교환':4, '반품':5, '결제':6

In [18]:
# 2중 리스트로 변환됨

data_list = []
for q, label in zip(df["msg"],df["cat"])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

## 3-1. Train / Test set 분리

* 라벨링은 이미 진행했으므로, 바로 train/ test 분리 진행

In [19]:
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [20]:
print(len(dataset_train))
print(len(dataset_test))

70761
23588


## 3-2. KoBERT 입력 데이터로 만들기

* 데이터를 train data와 test data로 나누었다면 각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등을 해주어야 한다

In [21]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [17]:
# Setting parameters

max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [22]:
# 토큰화
tokenizer= get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /home/adminuser/notebooks/modeling/question/[0차] KoBERT_cat_baseline/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [23]:
# 첫 번째는 패딩된 시퀀스
# 두 번째는 길이와 타입에 대한 내용
# 세 번재는 어텐션 마스크 시퀀스

data_train[0]

(array([   2, 3990, 7835, 2123, 5859, 3135, 4926, 4064, 5330, 3135,  517,
        6989, 5761, 6999,  517,   54,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(17, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [24]:
from transformers import AutoModel, AutoTokenizer


In [25]:
# pytorch용 DataLoader 사용(torch 형식의 dataset을 만들어주기)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## 3-3. KoBERT 학습모델 만들기

* 'AS':0, 주문':1, '배송':2, '업무처리':3, '교환':4, '반품':5, '결제':6
* 7가지의 class를 분류하기 때문에 num_classes는 7으로 입력

In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [27]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## 3-4. KoBERT 모델 학습시키기

In [28]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_20597/2673822008.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1106 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.0365653038024902 train acc 0.140625
epoch 1 batch id 201 loss 1.7634984254837036 train acc 0.23227611940298507
epoch 1 batch id 401 loss 1.3957653045654297 train acc 0.3008494389027431
epoch 1 batch id 601 loss 1.6216471195220947 train acc 0.3336366472545757
epoch 1 batch id 801 loss 1.5650694370269775 train acc 0.3514942259675406
epoch 1 batch id 1001 loss 1.303082823753357 train acc 0.36432317682317683
epoch 1 train acc 0.36994546101089404


/tmp/ipykernel_20597/2673822008.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/369 [00:00<?, ?it/s]

...

epoch 4 test acc 0.45825805480277026


  0%|          | 0/1106 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.1848827600479126 train acc 0.484375
epoch 5 batch id 201 loss 1.0970348119735718 train acc 0.5651430348258707
epoch 5 batch id 401 loss 1.0938646793365479 train acc 0.5737998753117207
epoch 5 batch id 601 loss 1.1241867542266846 train acc 0.5795549084858569
epoch 5 batch id 801 loss 1.0084482431411743 train acc 0.5792759051186017
epoch 5 batch id 1001 loss 1.0068126916885376 train acc 0.5811376123876124
epoch 5 train acc 0.582351185881886


  0%|          | 0/369 [00:00<?, ?it/s]

epoch 5 test acc 0.45754761367058117


In [29]:
import torch
import torchvision.models as models


#모델의 형태를 포함하여 저장하기
torch.save(model, 'KoBERT_cat_v.0.0.1_sona.pth')

#불러오기
# model = torch.load('model.pth')

## 3-5.새로운 문장 테스트

In [32]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            # 'AS':0, '주문':1, '배송':2, '업무처리':3, '교환':4, '반품':5, '결제':6

            if np.argmax(logits) == 0:
                test_eval.append("AS")
            elif np.argmax(logits) == 1:
                test_eval.append("주문")
            elif np.argmax(logits) == 2:
                test_eval.append("배송")
            elif np.argmax(logits) == 3:
                test_eval.append("업무처리")
            elif np.argmax(logits) == 4:
                test_eval.append("교환")
            elif np.argmax(logits) == 5:
                test_eval.append("반품")
            elif np.argmax(logits) == 6:
                test_eval.append("결제")

        print(">> 입력하신 질문은 " + test_eval[0] + "유형이라고 판단됩니다.")

using cached model. /home/adminuser/notebooks/modeling/question/[0차] KoBERT_cat_baseline/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [33]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 라지사이즈는 몇 cm인가요?
>> 입력하신 질문은 주문유형이라고 판단됩니다.


하고싶은 말을 입력해주세요 : 바꿔줄수있나요?
>> 입력하신 질문은 배송유형이라고 판단됩니다.


하고싶은 말을 입력해주세요 : 어떻게 고치나요?
>> 입력하신 질문은 업무처리유형이라고 판단됩니다.


하고싶은 말을 입력해주세요 : 환불되나요
>> 입력하신 질문은 반품유형이라고 판단됩니다.


하고싶은 말을 입력해주세요 : 카드취소해주세요
>> 입력하신 질문은 배송유형이라고 판단됩니다.


하고싶은 말을 입력해주세요 : 서비스는 센터 번호는 뭔가요?
>> 입력하신 질문은 배송유형이라고 판단됩니다.




KeyboardInterrupt: Interrupted by user